# xFLS master catalogue: Flags

In [1]:
import numpy as np
from astropy.table import Table

import itertools

from herschelhelp_internal.flagging import flag_outliers

In [2]:
SUFFIX = "20180501"
FIELD = "xFLS"
catname = "../../dmu1/dmu1_ml_xFLS/data/master_catalogue_xfls_{}.fits".format(SUFFIX)
master_catalogue = Table.read(catname)

In [3]:
g_bands = ["WFC g", "GPC1 g", "BASS g"]
r_bands = ["WFC r", "GPC1 r", "BASS r"]
z_bands = ["WFC z", "GPC1 z", "BASS z"]

all_bands = [g_bands, r_bands, z_bands]

## 1. Magnitudes and magnitude erros

In [4]:
def flag_mag(flagcol, mask):
    
    # Add flag columns if does not exist
    if flagcol not in master_catalogue.colnames:
        master_catalogue[flagcol] = np.zeros(len(master_catalogue), dtype=bool)
    
    # Flagged
    master_catalogue[flagcol][mask] = np.ones(len(mask), dtype=bool)
    print('    Number of flagged objects:', len(master_catalogue[flagcol][mask]))

### 1.a Pan-STARRS Aperture and Total magnitude errors

In [5]:
## dmu0: Pan-STARRS forced photometry catalogue
gpc1_err = 0.0010860000038519502
bands = ["GPC1 g", "GPC1 r", "GPC1 i", "GPC1 z", "GPC1 y"]

for i, band in enumerate(bands):
    print(band)
    basecol = band.replace(" ", "_").lower()
    
    ecol_ap, ecol_tot = "merr_ap_{}".format(basecol), "merr_{}".format(basecol)
    flagcol_ap, flagcol_tot = "flag_{}_ap".format(basecol), "flag_{}".format(basecol)
    
    mask_ap  = np.where(master_catalogue[ecol_ap]  == gpc1_err)[0]
    mask_tot = np.where(master_catalogue[ecol_tot] == gpc1_err)[0]
    
    print('  Aperture magnitude')
    flag_mag(flagcol_ap, mask_ap)
    print('  Total magnitude')
    flag_mag(flagcol_tot, mask_tot)
    

GPC1 g
  Aperture magnitude
    Number of flagged objects: 208
  Total magnitude
    Number of flagged objects: 216
GPC1 r
  Aperture magnitude
    Number of flagged objects: 252
  Total magnitude
    Number of flagged objects: 280
GPC1 i
  Aperture magnitude
    Number of flagged objects: 168
  Total magnitude
    Number of flagged objects: 173
GPC1 z
  Aperture magnitude
    Number of flagged objects: 117
  Total magnitude
    Number of flagged objects: 126
GPC1 y
  Aperture magnitude
    Number of flagged objects: 46
  Total magnitude
    Number of flagged objects: 48


### 2.c IRAC Aperture magnitudes

In [6]:
irac_mag = 3.9000000001085695
bands = ["IRAC i1", "IRAC i2", "IRAC i3", "IRAC i4"]

for i, band in enumerate(bands):
    print(band)
    basecol = band.replace(" ", "_").lower()
    
    ecol_ap = "merr_ap_{}".format(basecol)
    flagcol_ap = "flag_{}_ap".format(basecol)
    
    mask_ap  = np.where(master_catalogue[ecol_ap]  == irac_mag)[0]
    
    print('  Aperture magnitude')
    flag_mag(flagcol_ap, mask_ap)

IRAC i1
  Aperture magnitude
    Number of flagged objects: 0
IRAC i2
  Aperture magnitude
    Number of flagged objects: 0
IRAC i3
  Aperture magnitude
    Number of flagged objects: 0
IRAC i4
  Aperture magnitude
    Number of flagged objects: 0


## 2. Outliers

In [7]:
for band_of_a_kind in all_bands:
    for band1, band2 in itertools.combinations(band_of_a_kind, 2):
        #print(band1, band2)

        basecol1, basecol2 = band1.replace(" ", "_").lower(), band2.replace(" ", "_").lower()
        
        # Aperture mag
        col1, col2 = "m_ap_{}".format(basecol1), "m_ap_{}".format(basecol2)
        ecol1, ecol2 = "merr_ap_{}".format(basecol1), "merr_ap_{}".format(basecol2)
        flagcol1, flagcol2 = "flag_ap_{}".format(basecol1), "flag_ap_{}".format(basecol2)
        
        try:
            master_catalogue = flag_outliers(master_catalogue, col1, col2,
                      ecol1, ecol2,
                      flagcol1, flagcol2,
                      labels=("{} (aperture)".format(band1), "{} (aperture)".format(band2)))
        except KeyError:
            print("One of {} and {} is not present in the catalogue.".format(col1, col2))
                      
        
        # Tot mag
        col1, col2 = "m_{}".format(basecol1), "m_{}".format(basecol2)              
        ecol1, ecol2 = "merr_{}".format(basecol1), "merr_{}".format(basecol2)              
        flagcol1, flagcol2 = "flag_{}".format(basecol1), "flag_{}".format(basecol2)
              
        try:
            master_catalogue = flag_outliers(master_catalogue, col1, col2, 
                      ecol1, ecol2,
                      flagcol1, flagcol2,
                      labels=("{} (total)".format(band1), "{} (total)".format(band2)))   
        except KeyError:
            print("One of {} and {} is not present in the catalogue.".format(col1, col2))          
                      

GPC1 g (aperture) - WFC g (aperture):
  Number of outliers: 2
GPC1 g (total) - WFC g (total):
  Number of outliers: 0
One of m_ap_wfc_g and m_ap_bass_g is not present in the catalogue.
One of m_wfc_g and m_bass_g is not present in the catalogue.
One of m_ap_gpc1_g and m_ap_bass_g is not present in the catalogue.
One of m_gpc1_g and m_bass_g is not present in the catalogue.
GPC1 r (aperture) - WFC r (aperture):
  Number of outliers: 11
GPC1 r (total) - WFC r (total):
  Number of outliers: 4
One of m_ap_wfc_r and m_ap_bass_r is not present in the catalogue.
One of m_wfc_r and m_bass_r is not present in the catalogue.
One of m_ap_gpc1_r and m_ap_bass_r is not present in the catalogue.
One of m_gpc1_r and m_bass_r is not present in the catalogue.
GPC1 z (aperture) - WFC z (aperture):
  Number of outliers: 1
GPC1 z (total) - WFC z (total):
  Number of outliers: 0
One of m_ap_wfc_z and m_ap_bass_z is not present in the catalogue.
One of m_wfc_z and m_bass_z is not present in the catalogue.
O

## 3. Save table

In [8]:
flag_cols = ["help_id"]
for col in master_catalogue.colnames:
    if col.startswith("flag_"):
        flag_cols += [col]
new_catname = "./data/{}_{}_flags.fits".format(FIELD.lower(),SUFFIX)
master_catalogue[flag_cols].write(new_catname, overwrite = True)